In [1]:
# импорт всех необходимых для работы библиотек
import telebot
from telebot import types
import json
import glob
import sqlite3
import random
import time

# инициализация бота
bot = telebot.TeleBot("5447058268:AAH2PdXo10zugOd5LTHxm0Wwn8NkWnQGYX4")


# вспомогательный класс, который будет определять номер текущего продукта
class Step:
    def __init__(self):
        self.step = 0  # текущий шаг для определения блюда
        self.flag = True  # идентификатор первого/последнего сообщения
        self.msg = -1  # переменная для хранения информации о последнем сообщении для его редактирования

    # функция, увеличивающая шаг на 1
    def add(self, num):
        self.step += num

    # функция, умньшающая шаг на 1
    def back(self, num):
        self.step -= num

    # функция обновления
    def new_cycle(self):
        self.step = 0
        self.flag = True
        self.msg = -1


class Info:
    def __init__(self):
        self.name = 'No name'
        self.price = 0


# инициаилизация шага для каждой категории меню
zak = Step()
salad = Step()
hot = Step()
pasta = Step()
piz = Step()
mang = Step()
garn = Step()
bar = Step()
default = Step()  # для редактирования/удаления последних сообщений
about = Info()


# загрузка меню
with open('menu.json') as f:
    data = json.load(f)


# формируем таблицу для сохранения рецептов для каждого пользователя ботом
df = sqlite3.connect('cust_bag.db')
cursor = df.cursor()

try:
    query = "CREATE TABLE \"cust_bag\" (\
            \"random_id\" INTEGER UNIQUE, \
            \"user_id\" BIGINT, \
            \"Product\" TEXT, \
            \"Price\" TEXT, \
            PRIMARY KEY(\"random_id\"))"
    cursor.execute(query)
except:
    pass


# хэндлер обработки команды "старт"
@bot.message_handler(commands=['start'])
def get_message(mas):
    # на всякий случай будем обновлять параметры классов при вызове команды /start
    for elem in [zak, salad, hot, pasta, piz, mang, garn, bar]:
        elem.new_cycle()

    # создание внутренней клавиатуры
    menu = types.InlineKeyboardMarkup(row_width=1)
    item1 = types.InlineKeyboardButton("Ознакомиться с меню 👩‍🍳", callback_data='get_menu')
    item2 = types.InlineKeyboardButton("Корзина 🧺", callback_data='bag')
    item3 = types.InlineKeyboardButton("Общая информация о ресторане ℹ️", callback_data='info')

    menu.add(item1)
    menu.add(item2)
    menu.add(item3)

    # отправка сообщения пользователю:
    # если клиент зашел впервые, то он увидит первый вариант, если в результате переходов он вернется в меню,
    # то увидит второй вариант
    if default.msg == -1:
        default.msg = bot.send_message(chat_id=mas.chat.id,
                                       text='👋 Вас приветствует интерактивный помощник ресторана *Сцена*!\n\n'
                                            'Выберите действие 👇',
                                       parse_mode="Markdown",
                                       reply_markup=menu)
    else:
        default.msg = bot.send_message(chat_id=mas.chat.id,
                                       text='👋 Вновь приветствуем в меню интерактивного помощника ресторана '
                                            '*Сцена*!\n\n'
                                            'Выберите действие 👇',
                                       parse_mode="Markdown",
                                       reply_markup=menu)


# хэндлер обработки ответа с вирутальной клавиатуры (с главного экрана и др. страниц)
@bot.callback_query_handler(func=lambda call: call.data in ['get_menu', 'bag', 'info', 'main'])
def callback_inline(call):
    try:
        if call.message:
            if call.data == 'get_menu':
                show_menu(call)
            elif call.data == 'bag':
                cust_bag(call)
            elif call.data == 'info':
                get_info(call)
            elif call.data == 'main':
                bot.delete_message(chat_id=call.message.chat.id,
                                   message_id=default.msg.message_id)
                get_message(call.message)
            else:
                default.msg = bot.send_message(call.message.chat.id, text='Кажется что-то пошло не так :(\n'
                                                                          'Попробуйте заново с помощью /start')
    except Exception as e:
        print(e)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка :(\n'
                                                                  'Попробуйте заново с помощью /start')


# функция показа информации о ресторане
def get_info(call):
    try:
        text = '🥂 *Ресторан Сцена*\n\n' \
               '🧭 *Адрес:* ул. Новочеремушкинская, д. 21А, Москва\n\n' \
               '⏳ *Время работы:*\n' \
               '• Понедельник-Четверг, Воскресенье: с 11:00 до 23:00\n' \
               '• Пятница, Суббота: с 11:00 до 6:00\n\n' \
               '🎤 *Караоке:*\n' \
               '• Пятница, Суббота: с 23:00 до 6:00\n\n' \
               '📞 *Бронь и связь с администратором:* +7(985)350-70-70'

        # клавиатура с основными переходами из блока "Информация"
        info = types.InlineKeyboardMarkup(row_width=1)
        item1 = types.InlineKeyboardButton("Ознакомиться с меню 👩‍🍳", callback_data='get_menu')
        item2 = types.InlineKeyboardButton("Корзина 🧺", callback_data='bag')
        item3 = types.InlineKeyboardButton("Главная 🏠", callback_data='main')

        info.add(item1)
        info.add(item2)
        info.add(item3)

        bot.delete_message(chat_id=call.message.chat.id,
                           message_id=default.msg.message_id)

        default.msg = bot.send_photo(photo=open('map.png', 'rb'),
                                     chat_id=call.message.chat.id,
                                     caption=text,
                                     parse_mode='Markdown',
                                     reply_markup=info)

    except Exception as e:
        print(e)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка :(\n'
                                                                  'Попробуйте заново с помощью /start')


# функция показа корзины
def cust_bag(call):
    try:
        bag_key = types.InlineKeyboardMarkup(row_width=1)

        item1 = types.InlineKeyboardButton('Просмотреть всю корзину 🧺', callback_data='show_all')
        item2 = types.InlineKeyboardButton('Редактировать корзину 🛠', callback_data='edit')
        item3 = types.InlineKeyboardButton('Очистить корзину ❌', callback_data='clear')
        item4 = types.InlineKeyboardButton("Главная 🏠", callback_data='main')

        bag_key.add(item1)
        bag_key.add(item2)
        bag_key.add(item3)
        bag_key.add(item4)

        bot.delete_message(chat_id=call.message.chat.id, message_id=default.msg.message_id)

        default.msg = bot.send_message(call.from_user.id,
                                       text='Выберите действие 👇',
                                       reply_markup=bag_key)
    except Exception as e:
        print(e)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка c корзиной :(\n'
                                                                  'Попробуйте заново с помощью /start')


# хэндлер для обработки ответа с кнопок при работе с корзиной
@bot.callback_query_handler(func=lambda call: call.data in ['add_one', 'show_all', 'edit', 'clear'])
def callback_cust_bag(call):
    try:
        if call.message:
            if call.data == 'add_one':
                add_one_prod(call)
            elif call.data == 'show_all':
                show_all_prod(call)
            elif call.data == 'edit':
                edit_cust_bag(call)
            elif call.data == 'clear':
                clear_all(call)
            else:
                default.msg = bot.send_message(call.message.chat.id, text='Кажется что-то пошло не так :(\n'
                                                                          'Попробуйте заново с помощью /start')
    except Exception as e:
        print(e, 1)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка :(\n'
                                                                  'Попробуйте заново с помощью /start')


# функция добавления блюда в корзину
def add_one_prod(call):
    with sqlite3.connect('cust_bag.db') as frame:
        cursor = frame.cursor()
        random_id = random.randint(0, 1000000000)
        user_id = call.from_user.id
        cursor.execute("INSERT INTO cust_bag VALUES (?, ?, ?, ?)",
                       (random_id, user_id, about.name, about.price))
        frame.commit()

        # уведомление для клиента о том, что блюдо добавлено
        a = bot.send_message(chat_id=call.message.chat.id, text='Блюдо успешно добавлено в корзину 👏')
        time.sleep(0.75)
        bot.delete_message(chat_id=call.message.chat.id, message_id=a.message_id)


# функция обработки названий
def values(bludo):
    bludo_str = []
    s = 0
    maxi = 0
    for val in list(bludo):
        if '/' in str(val[1]):
            s += int(val[1].split('/')[2])
            price = val[1].split('/')[2]
        else:
            s += int(val[1])  # общий чек
            price = int(val[1])

        if len(f"• {val[0]} —— {s}₽\n") > maxi:
            maxi = len(f"• {val[0]} —— {price}₽\n")  # длина самого длинного сообщения (для красоты)

        bludo_str.append(f"• {val[0]} —— {price}₽\n")

    bludo_str.append(f"\n{'-' * maxi}")
    bludo_str.append(f"\n*Итого:* {s}₽")
    return ''.join(bludo_str), s


# функция показа всег содержимого в корзине
def show_all_prod(call):
    try:
        bag_key = types.InlineKeyboardMarkup(row_width=1)

        item1 = types.InlineKeyboardButton('Редактировать корзину 🛠', callback_data='edit')
        item2 = types.InlineKeyboardButton('Очистить корзину ❌', callback_data='clear')
        item3 = types.InlineKeyboardButton("Ознакомиться с меню 👩‍🍳", callback_data='get_menu')
        item4 = types.InlineKeyboardButton("Главная 🏠", callback_data='main')

        bag_key.add(item1)
        bag_key.add(item2)
        bag_key.add(item3)
        bag_key.add(item4)

        # если таблица непустая, клиент увидит свою корзину
        with sqlite3.connect('cust_bag.db') as frame:
            cursor = frame.cursor()
            cursor.execute(f"SELECT Product, Price FROM cust_bag WHERE user_id=={call.from_user.id}")
            bludo, check = values(cursor.fetchall())

            if check == 0:  # если клиент ничего не заказал, выводим соответствующую клавиатуру и текст
                bag_key = types.InlineKeyboardMarkup(row_width=1)

                item1 = types.InlineKeyboardButton("Ознакомиться с меню 👩‍🍳", callback_data='get_menu')
                item2 = types.InlineKeyboardButton("Главная 🏠", callback_data='main')

                bag_key.add(item1)
                bag_key.add(item2)

                # иначе будет сообщение о том, что корзина пустая
                bot.delete_message(chat_id=call.message.chat.id, message_id=default.msg.message_id)
                default.msg = bot.send_message(chat_id=call.message.chat.id, text='Здесь пусто. Добавьте блюдо 🧺',
                                               reply_markup=bag_key)
            else:  # иначе меняем немного текст и клавиатуру
                bot.delete_message(chat_id=call.message.chat.id, message_id=default.msg.message_id)
                default.msg = bot.send_message(chat_id=call.message.chat.id,
                                               text=bludo,
                                               parse_mode='Markdown',
                                               reply_markup=bag_key)
                frame.commit()
    except Exception as e:
        # данный except чаще всего будет срабатывать, если человек ничего не сложил в корзину, но решил ее открыть
        print(e)

        bag_key = types.InlineKeyboardMarkup(row_width=1)

        item1 = types.InlineKeyboardButton("Ознакомиться с меню 👩‍🍳", callback_data='get_menu')
        item2 = types.InlineKeyboardButton("Главная 🏠", callback_data='main')

        bag_key.add(item1)
        bag_key.add(item2)

        # иначе будет сообщение о том, что корзина пустая
        default.msg = bot.send_message(chat_id=call.message.chat.id, text='Здесь пусто. Добавьте блюдо 🧺',
                                       reply_markup=bag_key)


# функция анализа корзины с целью редактирования
def edit_cust_bag(call):
    with sqlite3.connect('cust_bag.db') as frame:

        bag_key = types.InlineKeyboardMarkup(row_width=1)

        item1 = types.InlineKeyboardButton("Ознакомиться с меню 👩‍🍳", callback_data='get_menu')
        item2 = types.InlineKeyboardButton("Главная 🏠", callback_data='main')

        bag_key.add(item1)
        bag_key.add(item2)

        cursor = frame.cursor()

        # Откроем все блюда в корзине пользователя
        cursor.execute(f'SELECT Product FROM cust_bag WHERE user_id=={call.from_user.id}')
        bludo = list(set(cursor.fetchall()))
        frame.commit()

        # NB: важно учитывать, что если клиент добавил несколько раз в корзину одно и то же блюдо,
        # то при удалении исчезнут ВСЕ упоминания об этом блюде

        # если в таблице пусто, сообщим об этом пользователю
        if len(bludo) == 0:
            bot.delete_message(chat_id=call.message.chat.id, message_id=default.msg.message_id)
            default.msg = bot.send_message(chat_id=call.message.chat.id,
                                           text='В корзине нечего удалять, ознакомьтесь с меню 👇',
                                           reply_markup=bag_key)

        # иначе предложим выбрать то блюдо, которое стоит удалить
        else:
            keyboard = types.ReplyKeyboardMarkup(row_width=1)

            for value in range(len(bludo)):
                item = bludo[value][0]
                keyboard.add(types.KeyboardButton(item))

            msg = bot.send_message(call.from_user.id,
                                   text="Выберите одно блюдо из списка 👇",
                                   reply_markup=keyboard)

            # перейдем в функцию, непосредственно выполняющую удаление выбранного блюда
            bot.register_next_step_handler(msg, worker_delete_one_plan)


# функция удаления выбранного блюда
def worker_delete_one_plan(msg):
    with sqlite3.connect('cust_bag.db') as frame:
        cursor = frame.cursor()
        thing = msg.text
        cursor.execute('DELETE FROM cust_bag WHERE user_id==? AND Product==?', (msg.from_user.id, thing))
        frame.commit()

    bag_key = types.InlineKeyboardMarkup(row_width=1)

    item1 = types.InlineKeyboardButton('Просмотреть всю корзину 🧺', callback_data='show_all')
    item2 = types.InlineKeyboardButton('Очистить корзину ❌', callback_data='clear')
    item3 = types.InlineKeyboardButton("Ознакомиться с меню 👩‍🍳", callback_data='get_menu')
    item4 = types.InlineKeyboardButton("Главная 🏠", callback_data='main')

    bag_key.add(item1)
    bag_key.add(item2)
    bag_key.add(item3)
    bag_key.add(item4)

    bot.delete_message(chat_id=default.msg.chat.id, message_id=default.msg.message_id)
    default.msg = bot.send_message(chat_id=msg.chat.id,
                                   text='Выбранный рецепт был удален 👏',
                                   reply_markup=bag_key)


# функция полной очистки корзины
def clear_all(call):
    try:
        # производим удаление
        with sqlite3.connect('cust_bag.db') as frame:
            cursor = frame.cursor()
            cursor.execute(f'DELETE FROM cust_bag WHERE user_id=={call.from_user.id}')
            frame.commit()

        bag_key = types.InlineKeyboardMarkup(row_width=1)

        item1 = types.InlineKeyboardButton("Ознакомиться с меню 👩‍🍳", callback_data='get_menu')
        item2 = types.InlineKeyboardButton("Главная 🏠", callback_data='main')

        bag_key.add(item1)
        bag_key.add(item2)

        bot.delete_message(chat_id=default.msg.chat.id, message_id=default.msg.message_id)
        default.msg = bot.send_message(chat_id=call.message.chat.id,
                                       text='Удалены все избранные блюда 👏',
                                       reply_markup=bag_key)
    except:
        # иначе пользователю будет сообщено, что ничего нет
        bag_key = types.InlineKeyboardMarkup(row_width=1)

        item1 = types.InlineKeyboardButton("Ознакомиться с меню 👩‍🍳", callback_data='get_menu')
        item2 = types.InlineKeyboardButton("Главная 🏠", callback_data='main')

        bag_key.add(item1)
        bag_key.add(item2)

        bot.delete_message(chat_id=default.msg.chat.id, message_id=default.msg.message_id)
        default.msg = bot.send_message(chat_id=call.message.chat.id,
                                       text='Удалены все избранные блюда 👏',
                                       reply_markup=bag_key)


# функция показа меню
def show_menu(call):
    global data
    eda = types.InlineKeyboardMarkup(row_width=1)

    # формируем клавиатуру под каждую категорию блюд
    for elem in data.keys():
        eda.add(types.InlineKeyboardButton(elem, callback_data=elem))

    eda.add(types.InlineKeyboardButton("Главная 🏠", callback_data='main'))

    default.msg = bot.delete_message(chat_id=call.message.chat.id, message_id=default.msg.message_id)
    default.msg = bot.send_message(chat_id=call.message.chat.id,
                                   text='Выберите категорию меню 👇',
                                   reply_markup=eda)


# функция показа блюда из выбранной категории меню
def show_menu_category(call, category, name, clas, next):
    try:
        i = clas.step  # текущий шаг
        if (clas.step != len(category)) or (clas.step < 0):
            # эсли это не первое и не последнее блюда, то обычный вид (иначе см. else ниже)

            src = glob.glob(f"Photo/{name}/{name.lower()}{i + 1}.*")[0]  # сбор картинки из папки

            product = category[i]['Название'].capitalize()  # название блюда
            compound = str.title(category[i]['Состав'])  # состав
            ans = []
            for elem in compound.split(', '):
                ans.append('• ' + str(elem).capitalize() + '\n')
            compound = ''.join(ans)  # подготовленный под вывод состав (добавлены точки и разбито по \n)
            price = category[i]['Цена']  # цена

            if clas.step == 0:  # если это первое блюда, то клавиатура будет немного другой (без предыдущего рецепта)
                menu_keyboard = types.InlineKeyboardMarkup(row_width=1)
                item1 = types.InlineKeyboardButton("Следующее блюдо 👉", callback_data=f'next_one_{next}')
                item2 = types.InlineKeyboardButton("Добавить в корзину 🛒", callback_data='add_one')
                item3 = types.InlineKeyboardButton("Назад в меню 🔙", callback_data='get_menu')
                item4 = types.InlineKeyboardButton("Главная 🏠", callback_data='main')

                menu_keyboard.add(item1)
                menu_keyboard.add(item2)
                menu_keyboard.add(item3)
                menu_keyboard.add(item4)

                text = f'*Название блюда ({i + 1} из {len(category)}):*\n'\
                       f'• {product}\n\n'\
                       f'*Состав блюда:* \n{compound}\n'\
                       f'*Цена блюда:*\n • {price}'

                # обновим атрибуты класса в случае, если понадобится записать в таблицу с корзиной
                about.name = product
                about.price = price.replace('₽', '')

            else:  # если же блюдо не первое, то добавляется кнопка "следующий рецепт"
                menu_keyboard = types.InlineKeyboardMarkup(row_width=1)
                item1 = types.InlineKeyboardButton("Следующее блюдо 👉", callback_data=f'next_one_{next}')
                item2 = types.InlineKeyboardButton("Предыдущее блюдо 👈", callback_data=f'back_one_{next}')
                item3 = types.InlineKeyboardButton("Добавить в корзину 🛒", callback_data='add_one')
                item4 = types.InlineKeyboardButton("Назад в меню ‍🔙", callback_data='get_menu')
                item5 = types.InlineKeyboardButton("Главная 🏠", callback_data='main')

                menu_keyboard.add(item1)
                menu_keyboard.add(item2)
                menu_keyboard.add(item3)
                menu_keyboard.add(item4)
                menu_keyboard.add(item5)

                about.name = product
                about.price = price.replace('₽', '')

                text = f'*Название блюда ({i + 1} из {len(category)}):*\n'\
                       f'• {product}\n\n' \
                       f'*Состав блюда:* \n{compound}\n' \
                       f'*Цена блюда:*\n • {price}'

            # атрибут flag нужен для определения что необходимо сделать с прыдыдущим сообщением:
            # удалить или заменить (true и false соотвественно)
            if clas.flag:
                clas.flag = False
                default.msg = bot.delete_message(chat_id=call.message.chat.id, message_id=default.msg.message_id)
                default.msg = bot.send_photo(chat_id=call.message.chat.id,
                                             photo=open(src, 'rb'),
                                             caption=text,
                                             reply_markup=menu_keyboard,
                                             parse_mode='Markdown')
            else:
                default.msg = bot.edit_message_media(media=types.InputMediaPhoto(open(src, 'rb')),
                                                     chat_id=call.message.chat.id,
                                                     message_id=default.msg.message_id,
                                                     reply_markup=menu_keyboard)
                default.msg = bot.edit_message_caption(caption=text,
                                                       chat_id=call.message.chat.id,
                                                       message_id=default.msg.message_id,
                                                       reply_markup=menu_keyboard,
                                                       parse_mode='Markdown')
        else:  # если же это последний рецепт, то будет выведено
            global data
            eda = types.InlineKeyboardMarkup(row_width=1)

            for elem in data.keys():
                eda.add(types.InlineKeyboardButton(elem, callback_data=elem))
            eda.add(types.InlineKeyboardButton("Главная 🏠", callback_data='main'))

            for elem in [zak, salad, hot, pasta, piz, mang, garn, bar]:
                elem.new_cycle()

            default.msg = bot.delete_message(chat_id=call.message.chat.id,
                                             message_id=default.msg.message_id)
            default.msg = bot.send_message(call.message.chat.id,
                                           text='Блюда закончились, выберите категорию меню 👇',
                                           reply_markup=eda)
    except Exception as e:
        print(e)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка :(\n'
                                                                  'Попробуйте заново с помощью /start')


# хэндлер обработки ответа с виртуальной клавиатуры о меню
@bot.callback_query_handler(func=lambda call: call.data in data.keys())
def callback_menu(call):
    try:
        if call.message:
            if call.data == "Закуски/Ассорти 🧆":
                show_menu_category(call, data["Закуски/Ассорти 🧆"], "Zakuzki", zak, 'zak')
            elif call.data == "Салаты 🥬":
                show_menu_category(call, data["Салаты 🥬"], "Salad", salad, 'salad')
            elif call.data == "Горячее 🍲":
                show_menu_category(call, data["Горячее 🍲"], "Hot", hot, 'hot')
            elif call.data == "Паста 🍝":
                show_menu_category(call, data["Паста 🍝"], "Pasta", pasta, 'pasta')
            elif call.data == "Пицца 🍕":
                show_menu_category(call, data["Пицца 🍕"], "Pizza", piz, 'piz')
            elif call.data == "Мангал 🔥":
                show_menu_category(call, data["Мангал 🔥"], "Mangal", mang, 'mang')
            elif call.data == "Гарниры 🍟":
                show_menu_category(call, data["Гарниры 🍟"], "Garniers", garn, 'garn')
            elif call.data == "Бар 🍹":
                show_menu_category(call, data["Бар 🍹"], "Bar", bar, 'bar')
            else:
                bot.send_message(call.message.chat.id, text='Кажется что-то пошло не так :(\n'
                                                            'Попробуйте заново с помощью /start')
    except Exception as e:
        print(e)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка :(\n'
                                                                  'Попробуйте заново с помощью /start')


# хэндлер перехода к новому рецепту закусок
@bot.callback_query_handler(func=lambda call: call.data in ['next_one_zak', 'back_one_zak'])
def next_product_zak(call):
    try:
        if call.message:
            if call.data == "next_one_zak":
                zak.add(1)
                show_menu_category(call, data["Закуски/Ассорти 🧆"], "Zakuzki", zak, 'zak')
            elif call.data == "back_one_zak":
                zak.back(1)
                show_menu_category(call, data["Закуски/Ассорти 🧆"], "Zakuzki", zak, 'zak')
            else:
                bot.send_message(call.message.chat.id, text='Кажется что-то пошло не так :(\n'
                                                            'Попробуйте заново с помощью /start')
        else:
            bot.send_message(call.message.chat.id, text='Сообщение пустое')
    except Exception as e:
        print(e)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка :(\n'
                                                                  'Попробуйте заново с помощью /start')


# хэндлер перехода к новому рецепту салатов
@bot.callback_query_handler(func=lambda call: call.data in ['next_one_salad', 'back_one_salad'])
def next_product_salad(call):
    try:
        if call.message:
            if call.data == "next_one_salad":
                salad.add(1)
                show_menu_category(call, data["Салаты 🥬"], "Salad", salad, 'salad')
            elif call.data == "back_one_salad":
                salad.back(1)
                show_menu_category(call, data["Салаты 🥬"], "Salad", salad, 'salad')
            else:
                bot.send_message(call.message.chat.id, text='Кажется что-то пошло не так :(\n'
                                                            'Попробуйте заново с помощью /start')
    except Exception as e:
        print(e)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка :(\n'
                                                                  'Попробуйте заново с помощью /start')


# хэндлер перехода к новому рецепту горячего блюда
@bot.callback_query_handler(func=lambda call: call.data in ['next_one_hot', 'back_one_hot'])
def next_product_hot(call):
    try:
        if call.message:
            if call.data == "next_one_hot":
                hot.add(1)
                show_menu_category(call, data["Горячее 🍲"], "Hot", hot, 'hot')
            elif call.data == "back_one_hot":
                hot.back(1)
                show_menu_category(call, data["Горячее 🍲"], "Hot", hot, 'hot')
            else:
                bot.send_message(call.message.chat.id, text='Кажется что-то пошло не так :(\n'
                                                            'Попробуйте заново с помощью /start')
    except Exception as e:
        print(e)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка :(\n'
                                                                  'Попробуйте заново с помощью /start')


# хэндлер перехода к новому рецепту пасты
@bot.callback_query_handler(func=lambda call: call.data in ['next_one_pasta', 'back_one_pasta'])
def next_product_pasta(call):
    try:
        if call.message:
            if call.data == "next_one_pasta":
                pasta.add(1)
                show_menu_category(call, data["Паста 🍝"], "Pasta", pasta, 'pasta')
            elif call.data == "back_one_pasta":
                pasta.back(1)
                show_menu_category(call, data["Паста 🍝"], "Pasta", pasta, 'pasta')
            else:
                bot.send_message(call.message.chat.id, text='Кажется что-то пошло не так :(\n'
                                                            'Попробуйте заново с помощью /start')
    except Exception as e:
        print(e)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка :(\n'
                                                                  'Попробуйте заново с помощью /start')


# хэндлер перехода к новому рецепту пиццы
@bot.callback_query_handler(func=lambda call: call.data in ['next_one_piz', 'back_one_piz'])
def next_product_pizza(call):
    try:
        if call.message:
            if call.data == "next_one_piz":
                piz.add(1)
                show_menu_category(call, data["Пицца 🍕"], "Pizza", piz, 'piz')
            elif call.data == "back_one_piz":
                piz.back(1)
                show_menu_category(call, data["Пицца 🍕"], "Pizza", piz, 'piz')
            else:
                bot.send_message(call.message.chat.id, text='Кажется что-то пошло не так :(\n'
                                                            'Попробуйте заново с помощью /start')
    except Exception as e:
        print(e)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка :(\n'
                                                                  'Попробуйте заново с помощью /start')


# хэндлер перехода к новому рецепту мангала
@bot.callback_query_handler(func=lambda call: call.data in ['next_one_mang', 'back_one_mang'])
def next_product_mang(call):
    try:
        if call.message:
            if call.data == "next_one_mang":
                mang.add(1)
                show_menu_category(call, data["Мангал 🔥"], "Mangal", mang, 'mang')
            elif call.data == "back_one_mang":
                mang.back(1)
                show_menu_category(call, data["Мангал 🔥"], "Mangal", mang, 'mang')
            else:
                bot.send_message(call.message.chat.id, text='Кажется что-то пошло не так :(\n'
                                                            'Попробуйте заново с помощью /start')
    except Exception as e:
        print(e)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка :(\n'
                                                                  'Попробуйте заново с помощью /start')


# хэндлер перехода к новому рецепту гарниров
@bot.callback_query_handler(func=lambda call: call.data in ['next_one_garn', 'back_one_garn'])
def next_product_garn(call):
    try:
        if call.message:
            if call.data == "next_one_garn":
                garn.add(1)
                show_menu_category(call, data["Гарниры 🍟"], "Garniers", garn, 'garn')
            elif call.data == "back_one_garn":
                garn.back(1)
                show_menu_category(call, data["Гарниры 🍟"], "Garniers", garn, 'garn')
            else:
                bot.send_message(call.message.chat.id, text='Кажется что-то пошло не так :(\n'
                                                            'Попробуйте заново с помощью /start')
    except Exception as e:
        print(e)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка :(\n'
                                                                  'Попробуйте заново с помощью /start')


# хэндлер перехода к новому рецепту бара
@bot.callback_query_handler(func=lambda call: call.data in ['next_one_bar', 'back_one_bar'])
def next_product_bar(call):
    try:
        if call.message:
            if call.data == "next_one_bar":
                bar.add(1)
                show_menu_category(call, data["Бар 🍹"], "Bar", bar, 'bar')
            elif call.data == "back_one_bar":
                bar.back(1)
                show_menu_category(call, data["Бар 🍹"], "Bar", bar, 'bar')
            else:
                bot.send_message(call.message.chat.id, text='Кажется что-то пошло не так :(\n'
                                                            'Попробуйте заново с помощью /start')
    except Exception as e:
        print(e)
        default.msg = bot.send_message(call.message.chat.id, text='Кажется произошла ошибка :(\n'
                                                                  'Попробуйте заново с помощью /start')


bot.polling(none_stop=True)
